In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import cv2
import collections
import pickle
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss, accuracy_score, confusion_matrix
import seaborn as sns

In [2]:
train_data = []

In [3]:
with open('new_samples_authentic.pkl','rb') as file:
    train_authentic= pickle.load(file)
with open('new_samples_copy_moved.pkl', 'rb') as file:
    train_copy=pickle.load(file)
with open('new_samples_spliced.pkl', 'rb') as file:
    train_spliced=pickle.load(file)

In [4]:
len(train_authentic), len(train_copy), len(train_spliced)

(13446, 13037, 24128)

In [5]:
num_samples_per_class = 13037

train_data = random.sample(train_authentic, num_samples_per_class) + \
                random.sample(train_copy, num_samples_per_class) + \
                random.sample(train_spliced, num_samples_per_class)
train_labels = [0] * num_samples_per_class + [1] * num_samples_per_class + [2] * num_samples_per_class

In [68]:
# train_labels = [0]*13446 + [1]*13037 + [2]*24128

In [69]:
# train_labels = [0]*13446 + [1]*(24128+13037)

In [8]:
len(train_data)

39111

In [7]:
len(train_labels)

39111

In [9]:
x_train, x_cv, y_train, y_cv = train_test_split(train_data, train_labels, test_size=0.25, stratify=train_labels)

In [10]:
from keras.utils import to_categorical

y_train = to_categorical(y_train, num_classes=3)
y_cv = to_categorical(y_cv, num_classes=3)
# y_test = to_categorical(test_labels, num_classes=3)

In [11]:
del train_data

In [30]:
# # This function plots the confusion matrices given y_i, y_i_hat.
# def plot_confusion_matrix(test_y, predict_y):
#     C = confusion_matrix(test_y, predict_y)
#     # C = 9,9 matrix, each cell (i,j) represents number of points of class i are predicted class j
#
#     A =(((C.T)/(C.sum(axis=1))).T)
#     #divid each element of the confusion matrix with the sum of elements in that column
#
#     # C = [[1, 2],
#     #     [3, 4]]
#     # C.T = [[1, 3],
#     #        [2, 4]]
#     # C.sum(axis = 1)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
#     # C.sum(axix =1) = [[3, 7]]
#     # ((C.T)/(C.sum(axis=1))) = [[1/3, 3/7]
#     #                           [2/3, 4/7]]
#
#     # ((C.T)/(C.sum(axis=1))).T = [[1/3, 2/3]
#     #                           [3/7, 4/7]]
#     # sum of row elements = 1
#
#     B =(C/C.sum(axis=0))
#     #divid each element of the confusion matrix with the sum of elements in that row
#     # C = [[1, 2],
#     #     [3, 4]]
#     # C.sum(axis = 0)  axis=0 corresonds to columns and axis=1 corresponds to rows in two diamensional array
#     # C.sum(axix =0) = [[4, 6]]
#     # (C/C.sum(axis=0)) = [[1/4, 2/6],
#     #                      [3/4, 4/6]]
#     plt.figure(figsize=(20,4))
#
#     labels = [0,1]
#     # representing A in heatmap format
#     cmap=sns.light_palette("blue")
#
#     plt.subplot(1, 3, 2)
#     sns.heatmap(B, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
#     plt.xlabel('Predicted Class')
#     plt.ylabel('Original Class')
#     plt.title("Precision matrix")
#
#     plt.subplot(1, 3, 3)
#     # representing B in heatmap format
#     sns.heatmap(A, annot=True, cmap=cmap, fmt=".3f", xticklabels=labels, yticklabels=labels)
#     plt.xlabel('Predicted Class')
#     plt.ylabel('Original Class')
#     plt.title("Recall matrix")
#     plt.show()

# CNN Models

### ResNet50

In [12]:
import keras
from keras.models import load_model, Sequential
from keras.layers import Dense, Flatten, Dropout
import keras.backend as K
from keras import optimizers
from keras.layers import LeakyReLU
from keras.applications.resnet50 import ResNet50

In [25]:
resnet_model=ResNet50(weights='imagenet', include_top=False, input_shape=(64,64, 3))

model_aug=Sequential()
model_aug.add(resnet_model)

top_model=Sequential()
top_model.add(Flatten(input_shape=(2, 2, 2048)))

top_model.add(Dense(64, activation='relu'))
model_aug.add(Dropout(0.2))

top_model.add(Dense(3, activation='softmax'))

model_aug.add(top_model)

for layer in model_aug.layers[0].layers[:171]:
    layer.trainable=False

# model_aug.load_weights('resnet50_weights_1.h5')

In [26]:
model_aug.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 2, 2, 2048)        23587712  
                                                                 
 dropout_3 (Dropout)         (None, 2, 2, 2048)        0         
                                                                 
 sequential_7 (Sequential)   (None, 3)                 524547    
                                                                 
Total params: 24112259 (91.98 MB)
Trainable params: 1579267 (6.02 MB)
Non-trainable params: 22532992 (85.96 MB)
_________________________________________________________________


In [27]:
model_aug.compile(loss='categorical_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])

In [28]:
model_aug.fit(
    np.array(x_train),
    np.array(y_train),
    epochs = 10,
    validation_data=(np.array(x_cv), np.array(y_cv)),
    use_multiprocessing=True,
    batch_size=64
)

Epoch 1/10
459/459 [==============================] - 323s 697ms/step - loss: 1.0941 - accuracy: 0.3908 - val_loss: 1.1178 - val_accuracy: 0.3665
Epoch 2/10
459/459 [==============================] - 329s 718ms/step - loss: 1.0709 - accuracy: 0.4143 - val_loss: 1.1457 - val_accuracy: 0.3364
Epoch 3/10
459/459 [==============================] - 334s 729ms/step - loss: 1.0638 - accuracy: 0.4231 - val_loss: 1.1587 - val_accuracy: 0.3599
Epoch 4/10
459/459 [==============================] - 337s 734ms/step - loss: 1.0581 - accuracy: 0.4285 - val_loss: 1.0712 - val_accuracy: 0.4201
Epoch 5/10
459/459 [==============================] - 346s 754ms/step - loss: 1.0532 - accuracy: 0.4341 - val_loss: 1.0666 - val_accuracy: 0.4187
Epoch 6/10
459/459 [==============================] - 348s 759ms/step - loss: 1.0487 - accuracy: 0.4418 - val_loss: 1.2107 - val_accuracy: 0.3619
Epoch 7/10
459/459 [==============================] - 340s 740ms/step - loss: 1.0458 - accuracy: 0.4482 - val_loss: 1.0762 -

In [29]:
model_aug.save_weights('multiclass_resnet50_weights.h5')

In [24]:
with open('test_images_modified.pkl','rb') as file:
    test_authentic, test_copy_images, test_copy_masks, test_spliced_images, test_spliced_masks = pickle.load(file)

In [41]:
import cv2

def resize_images(images_list, target_size):
    resized_images = []
    for img in images_list:
        # Determine if the image needs to be enlarged or shrinked
        if img.shape[0] < target_size[0] or img.shape[1] < target_size[1]:
            interpolation = cv2.INTER_LINEAR  # Enlarging
        else:
            interpolation = cv2.INTER_AREA  # Shrinking

        # Resize the image with the appropriate interpolation method
        resized_img = cv2.resize(img, target_size, interpolation=interpolation)
        resized_images.append(resized_img)

    return resized_images

# Example usage
# Assuming you have a list of images called 'image_list'
resized_test_authentic = resize_images(test_authentic, (64,64))
resized_test_copy_images = resize_images(test_copy_images, (64,64))
# resized_test_copy_masks = resize_images(test_copy_masks, (64,64))
resized_test_spliced_images = resize_images(test_spliced_images, (64,64))
# resized_test_spliced_masks = resize_images(test_spliced_masks, (64,64))

In [42]:
resized_test_spliced_images[0].shape

(64, 64, 3)

In [43]:
resized_test_data= []
resized_test_data+=resized_test_authentic+resized_test_spliced_images+resized_test_copy_images
print(len(resized_test_data))

498


In [30]:
test_labels = [0]*166 + [1]*(2*166)

In [31]:
print(len(test_labels))

498


In [53]:
model_preds = []
for i in np.array(resized_test_data):
    pred = model_aug.predict(i[np.newaxis, ...])
    if pred<0.5:
        pred=0
    elif pred>=0.5:
        pred = 1
    model_preds.append(pred)

1/1 [==============================] - 0s 40ms/step


In [54]:
from sklearn.metrics import accuracy_score

score = accuracy_score(test_labels, model_preds)
print(score)

0.6666666666666666


In [44]:
model_aug.evaluate(np.array(resized_test_data), np.array(test_labels))

16/16 [==============================] - 3s 211ms/step - loss: 0.6374 - accuracy: 0.6667


[0.6373848915100098, 0.6666666865348816]

In [52]:
len(resized_test_data)

498

### learnable resizer

In [19]:
from tensorflow.keras import layers
from tensorflow import keras
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

In [20]:
INP_SIZE = (256, 384)
TARGET_SIZE = (64, 64)
INTERPOLATION = "bilinear"

AUTO = tf.data.AUTOTUNE
BATCH_SIZE = 64
EPOCHS = 5

In [21]:
def conv_block(x, filters, kernel_size, strides, activation=layers.LeakyReLU(0.2)):
    x = layers.Conv2D(filters, kernel_size, strides, padding="same", use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    if activation:
        x = activation(x)
    return x


def res_block(x):
    inputs = x
    x = conv_block(x, 16, 3, 1)
    x = conv_block(x, 16, 3, 1, activation=None)
    return layers.Add()([inputs, x])


def get_learnable_resizer(filters=16, num_res_blocks=1, interpolation=INTERPOLATION):
    inputs = layers.Input(shape=[None, None, 3])

    # First, perform naive resizing.
    naive_resize = layers.Resizing(
        *TARGET_SIZE, interpolation=interpolation
    )(inputs)

    # First convolution block without batch normalization.
    x = layers.Conv2D(filters=filters, kernel_size=7, strides=1, padding="same")(inputs)
    x = layers.LeakyReLU(0.2)(x)

    # Second convolution block with batch normalization.
    x = layers.Conv2D(filters=filters, kernel_size=1, strides=1, padding="same")(x)
    x = layers.LeakyReLU(0.2)(x)
    x = layers.BatchNormalization()(x)

    # Intermediate resizing as a bottleneck.
    bottleneck = layers.Resizing(
        *TARGET_SIZE, interpolation=interpolation
    )(x)

    # Residual passes.
    for _ in range(num_res_blocks):
        x = res_block(bottleneck)

    # Projection.
    x = layers.Conv2D(
        filters=filters, kernel_size=3, strides=1, padding="same", use_bias=False
    )(x)
    x = layers.BatchNormalization()(x)

    # Skip connection.
    x = layers.Add()([bottleneck, x])

    # Final resized image.
    x = layers.Conv2D(filters=3, kernel_size=7, strides=1, padding="same")(x)
    final_resize = layers.Add()([naive_resize, x])

    return tf.keras.Model(inputs, final_resize, name="learnable_resizer")


learnable_resizer = get_learnable_resizer()

In [34]:
x = learnable_resizer(resized_test_data[0])

ValueError: Exception encountered when calling layer 'learnable_resizer' (type Functional).

Input 0 of layer "conv2d" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (256, 384, 3)

Call arguments received by layer 'learnable_resizer' (type Functional):
  • inputs=tf.Tensor(shape=(256, 384, 3), dtype=float32)
  • training=None
  • mask=None

In [38]:
def get_model():
    backbone = model_aug
    inputs = layers.Input((INP_SIZE[0], INP_SIZE[1], 3))
    x = layers.Rescaling(scale=1.0 / 255)(inputs)
    x = learnable_resizer(x)
    outputs = backbone(x)

    return tf.keras.Model(inputs, outputs)

In [40]:
testing_model = get_model()
testing_model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(learning_rate=1e-3), metrics=['accuracy'])
testing_model.evaluate(np.array(resized_test_data), np.array(test_labels))

16/16 [==============================] - 10s 498ms/step - loss: 1.0335 - accuracy: 0.3333


[1.0334608554840088, 0.3333333432674408]

## EfficientNetV2B0

In [14]:
from tensorflow.keras.applications import EfficientNetV2B0

model_eff = EfficientNetV2B0(input_shape=(64,64,3), include_top=False, weights = 'imagenet')
for layer in model_eff.layers:
    layer.trainable=False

24274472/24274472 [==============================] - 4s 0us/step


In [20]:
from tensorflow.keras.optimizers import RMSprop

x = Sequential()
x.add(model_eff)
x.add(Flatten())
x.add(Dense(1024, activation='relu'))
x.add(Dropout(0.2))
x.add(Dense(1, activation='sigmoid'))

x.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 efficientnetv2-b0 (Functio  (None, 2, 2, 1280)        5919312   
 nal)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 5120)              0         
                                                                 
 dense_4 (Dense)             (None, 1024)              5243904   
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0         
                                                                 
 dense_5 (Dense)             (None, 1)                 1025      
                                                                 
Total params: 11164241 (42.59 MB)
Trainable params: 5244929 (20.01 MB)
Non-trainable params: 5919312 (22.58 MB)
________

In [22]:
x.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss= 'binary_crossentropy', metrics=['acc'])

In [23]:
x.fit(np.array(x_train),
      np.array(y_train),
      validation_data=(np.array(x_cv),np.array(y_cv)),
      epochs = 25,
      use_multiprocessing=True,
      verbose=2
      )

Epoch 1/25
4713/4713 - 732s - loss: 0.6001 - acc: 0.7212 - val_loss: 0.5889 - val_acc: 0.7226 - 732s/epoch - 155ms/step
Epoch 2/25
4713/4713 - 728s - loss: 0.5918 - acc: 0.7226 - val_loss: 0.5894 - val_acc: 0.7226 - 728s/epoch - 155ms/step
Epoch 3/25
4713/4713 - 730s - loss: 0.5903 - acc: 0.7226 - val_loss: 0.5907 - val_acc: 0.7226 - 730s/epoch - 155ms/step
Epoch 4/25
4713/4713 - 729s - loss: 0.5905 - acc: 0.7226 - val_loss: 0.5905 - val_acc: 0.7226 - 729s/epoch - 155ms/step
Epoch 5/25
4713/4713 - 734s - loss: 0.5905 - acc: 0.7226 - val_loss: 0.5905 - val_acc: 0.7226 - 734s/epoch - 156ms/step
Epoch 6/25
4713/4713 - 726s - loss: 0.5905 - acc: 0.7226 - val_loss: 0.5905 - val_acc: 0.7226 - 726s/epoch - 154ms/step
Epoch 7/25
4713/4713 - 722s - loss: 0.5906 - acc: 0.7226 - val_loss: 0.5905 - val_acc: 0.7226 - 722s/epoch - 153ms/step
Epoch 8/25
4713/4713 - 723s - loss: 0.5905 - acc: 0.7226 - val_loss: 0.5905 - val_acc: 0.7226 - 723s/epoch - 153ms/step
Epoch 9/25
4713/4713 - 722s - loss: 0.59

KeyboardInterrupt: 

In [110]:
x.evaluate(np.array(test_data), np.array(test_labels))

626/626 [==============================] - 113s 177ms/step - loss: 0.5730 - accuracy: 0.7258


[0.5729516744613647, 0.7258056402206421]

### testing the resnet50 model with old samples without resizing training images:

In [ ]:
model_aug.predict(np.array(resized_test_authentic[0]))

In [114]:
import cv2

def resize_images(images_list, target_size):
    resized_images = []
    for img in images_list:
        interpolation = cv2.INTER_AREA  # Shrinking
        # Resize the image with the appropriate interpolation method
        resized_img = cv2.resize(img, target_size, interpolation=interpolation)
        resized_images.append(resized_img)

    return resized_images

# Example usage
# Assuming you have a list of images called 'image_list'
resized_full_test = resize_images(full_testing_data, (64,64))

In [119]:
full_test_labels = [0]*166 + [1]*(2*166)

In [120]:
model_aug.evaluate(np.array(resized_full_test), np.array(full_test_labels))

16/16 [==============================] - 3s 177ms/step - loss: 0.7650 - accuracy: 0.6526


[0.7650363445281982, 0.6526104211807251]

In [135]:
model_aug.predict(np.array([resized_full_test[245]]))

1/1 [==============================] - 0s 42ms/step


array([[0.6711995]], dtype=float32)